In [2]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchquantum as tq

from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import random
import cv2

from skimage import io
from PIL import Image


In [3]:
# define controlled hadamard gate
sq2 = 1/np.sqrt(2)
def controlled_H(qdev, target,control):
      qdev.apply(tq.QubitUnitary(
      has_params=True,init_params=([[1,0,0,0],[0,sq2,0,sq2],[0,0,1,0],[0,sq2,0,-sq2]]),wires=[target,control]))
 

In [100]:
class QuanvolutionFilter(tq.QuantumModule):
  # the __init__ method initializes the quantum device, the general encoder,
  # a random quantum layer, and a measurement operator.
    def __init__(self):
        super().__init__()
        self.n_wires = 4  # two ancillas
        self.q_device = tq.QuantumDevice(n_wires=self.n_wires)
        # encoding the input data
        self.encoder = tq.GeneralEncoder(
        [   {'input_idx': [0], 'func': 'ry', 'wires': [0]},
            {'input_idx': [1], 'func': 'ry', 'wires': [1]},
            {'input_idx': [2], 'func': 'ry', 'wires': [2]},
            {'input_idx': [3], 'func': 'ry', 'wires': [3]},])
        


        # random circuit layer 
        self.q_layer = tq.RandomLayer(n_ops=8, wires=list(range(self.n_wires)))
        self.measure = tq.MeasureAll(tq.PauliZ)
        #self.expval = tq.expval()
    
# x has the dimension of (batch_size, 28, 28) representing a batch of greyscale images
#The method first reshapes the input data into a 2D array of shape 
#(batch_size, 784) by concatenating adjacent 2x2 blocks of pixels.
# data is the new reshaped tensor 
    def forward(self, x, use_qiskit=False):
        bsz = x.shape[0] # batch size
        size = 256 # height and width of an image
        x = x.view(bsz, size, size) # view all data 

        data_list = []

        for c in range(0, size, 2):
            for r in range(0, size, 2):
                data = torch.transpose(torch.cat((x[:, c, r], x[:, c, r+1], x[:, c+1, r], x[:, c+1, r+1])).view(4, bsz), 0, 1)
                if use_qiskit:
                    data = self.qiskit_processor.process_parameterized(
                        self.q_device, self.encoder, self.q_layer, self.measure, data)
                else:
                    self.encoder(self.q_device, data)
                    
                    #haar wavelet
                    # level 1 
                    self.q_device.h(wires = 3) 
                    self.q_device.swap([3,2])
                    self.q_device.swap([2,1])
                    self.q_device.swap([1,0])

                    # level 2 
                    controlled_H(self.q_device, target=2, control= 3)
                    self.q_device.cswap([3,2,1])
                    self.q_device.cswap([3,1,0])

                    # level 3
                    
                    #self.q_device.ccx([2,3,4])
                    #controlled_H(self.q_device, target=1, control= 4)
                    #self.q_device.ccx([2,3,4])
                    #perm
                    #self.q_device.ccx([2,3,4])
                    #self.q_device.cswap([4,1,0])
                    #self.q_device.ccx([2,3,4])

                    #level 4
                    #self.q_device.ccx([2,3,4])
                    #self.q_device.ccx([1,4,5])
                    #controlled_H(self.q_device, target=0, control= 5)
                    #self.q_device.ccx([1,4,5])
                    #self.q_device.ccx([2,3,4])



                    self.q_layer(self.q_device)
                    data = self.measure(self.q_device)
                    
                    #for i in range(4):
                    #    measure_result = []
                    #    measure_result.append(tq.expval(self.q_device,wires=i, observables= tq.PauliZ(wires=i)))

                    #    data = torch.tensor([measure_result])

                    
                data_list.append(data.view(bsz, 4)) # only keep the first 4 qubits
        
        result = torch.cat(data_list, dim=1).float()
        
        return result


In [101]:

class HybridModel(torch.nn.Module): 
    def __init__(self): 
        super().__init__() 
        self.qf = QuanvolutionFilter()
        self.linear = torch.nn.Linear(4*128*128, 2) 
    def forward(self, x, use_qiskit=False):
        with torch.no_grad():
          x = self.qf(x, use_qiskit)
        x = self.linear(x)
        return F.log_softmax(x, -1) 
    # F.log_softmax is the log of the softmax function, which is a common choice for the output of a classification model.
    
class HybridModel_without_qf(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(256*256, 2)
    
    def forward(self, x, use_qiskit=False):
        x = x.view(-1, 256*256)
        x = self.linear(x)
        return F.log_softmax(x, -1)

In [102]:
# importing the dataset 
class Oral_Can_Data(Dataset):
    '''Oral cancer dataset'''
    def __init__(self, csv_file, root_dir,transform = None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations) # returns the number of samples in the dataset
    
    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()

        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        image = torch.tensor([image])
        can_type = self.annotations.iloc[index, 1] # labels:  1 for non-cancerous, 2 for cancerous
        can_type = torch.tensor([can_type])
        can_type = can_type

    
        sample = {'image': image, 'can_type': can_type}
        
        for index in range(len(self.annotations)):
            sample

        if self.transform:
            image = self.transform(image)
        
        return (sample) # returns the image and the label


In [83]:
root_dir = '/home/iisers/Documents/oral_cancer_project/Combined_data'
output_dir = '/home/iisers/Documents/oral_cancer_project/Combined_data_resized'
csv_file= '/home/iisers/Documents/oral_cancer_project/labels .csv'

target_size = (256, 256)

annotations = pd.read_csv(csv_file)
index = 0
img_path = os.path.join(output_dir, annotations.iloc[index, 0])

image = io.imread(img_path)
image = torch.tensor([image])

#plt.imshow(image, cmap='gray')
print(len(annotations))
#for i in range(527):
#    can_type = torch.tensor(annotations.iloc[i, 1])
#    print(can_type)
can_type = torch.tensor(annotations.iloc[index, 1])
c = []
for index in range(2):
    c.append({'img': image, 'label': can_type})

print(c[0]['img'].shape)

527
torch.Size([1, 256, 256])


In [84]:
image

tensor([[[131, 142, 152,  ..., 247, 241, 255],
         [131, 125, 140,  ..., 248, 253, 249],
         [150, 117, 138,  ..., 254, 255, 255],
         ...,
         [255, 255, 255,  ..., 186, 196, 219],
         [255, 255, 255,  ..., 223, 195, 203],
         [253, 253, 253,  ..., 190, 202, 216]]], dtype=torch.uint8)

In [81]:
image[0][0]

tensor([131, 142, 152, 147, 193, 155, 150, 172, 204, 197, 172, 168, 174, 180,
        172, 170, 170, 174, 168, 172, 171, 154, 169, 158, 149, 148, 144, 157,
        144, 138, 150, 149, 144, 146, 138, 135, 133, 138, 137, 150, 141, 132,
        153, 150, 148, 157, 165, 158, 150, 171, 189, 134, 168, 162, 157, 156,
        151, 140, 150, 174, 152, 144, 142, 125, 169, 161, 133, 138, 185, 147,
        130, 155, 147, 123, 150, 144, 157, 151, 148, 132, 103, 116, 162, 166,
        134, 155, 168, 237, 243, 218, 149, 194, 243, 227, 193, 236, 235, 207,
        224, 219, 214, 218, 235, 223, 146, 143,  94,  94, 123, 127, 113, 145,
        143, 151, 158, 139, 140, 172, 159, 164, 157, 163, 151, 139, 141, 142,
        157, 146, 139, 121, 131, 142, 129, 131, 149, 122, 125, 163, 165, 166,
        164, 169, 183, 182, 179, 184, 176, 160, 154, 158, 151, 166, 185, 166,
        169, 174, 168, 163, 173, 148, 158, 162, 158, 150, 132, 142, 164, 170,
        169, 169, 171, 170, 162, 145, 154, 166, 170, 154, 142, 1

In [24]:
for i in train_set:
    print(i)

{'image': array([[153, 113, 122, ..., 175, 176, 176],
       [119, 113, 117, ..., 176, 176, 175],
       [123, 106, 119, ..., 176, 176, 175],
       ...,
       [173, 174, 178, ..., 110, 120, 119],
       [172, 177, 177, ..., 132, 126, 118],
       [175, 176, 174, ..., 117, 112,  70]], dtype=uint8), 'can_type': tensor([2])}
{'image': array([[154, 139, 122, ..., 205, 204, 203],
       [167, 158, 126, ..., 204, 204, 203],
       [139, 120, 127, ..., 205, 205, 204],
       ...,
       [209, 209, 210, ..., 151, 132, 116],
       [210, 210, 210, ..., 119, 112, 123],
       [211, 211, 210, ..., 129, 143, 166]], dtype=uint8), 'can_type': tensor([2])}
{'image': array([[255, 255, 255, ..., 218, 182, 177],
       [255, 255, 255, ..., 233, 196, 159],
       [255, 255, 255, ..., 235, 204, 180],
       ...,
       [100, 119, 113, ...,  97, 148, 152],
       [ 95, 109, 133, ..., 126, 148, 126],
       [134, 121, 128, ..., 142, 151, 150]], dtype=uint8), 'can_type': tensor([2])}
{'image': array([[199,

In [86]:
# loading the dataset
dataset = Oral_Can_Data(csv_file= '/home/iisers/Documents/oral_cancer_project/labels .csv',root_dir='/home/iisers/Documents/oral_cancer_project/Combined_data_resized')

train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_set, val_set,  test_set = torch.utils.data.random_split(dataset, [train_size,val_size,test_size])

sampler = torch.utils.data.RandomSampler(train_set + val_set + test_set, replacement=True, num_samples=train_size)
train_loader = dict()

train_loader = torch.utils.data.DataLoader(dataset=train_set + val_set + test_set, batch_size=64, sampler=sampler, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_set, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=64, shuffle=True)

dataflow = dict({'train' : train_set, 'valid': val_set , 'test' : test_set})

In [33]:
train_set + val_set + test_set

In [40]:
for i in dataflow:
    print(i, len(dataflow[i]))

train 368
valid 79
test 80


In [63]:
for i  in dataflow['train']:
    print(i)

/tmp/ipykernel_3300/2295154719.py:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  image = torch.tensor([[image]])


{'image': tensor([[[[201, 182, 191,  ..., 254, 254, 254],
          [181, 159, 171,  ..., 254, 254, 254],
          [153, 203, 197,  ..., 254, 254, 254],
          ...,
          [234, 238, 200,  ..., 192, 205, 191],
          [247, 221, 202,  ..., 186, 202, 187],
          [218, 189, 193,  ..., 189, 191, 189]]]], dtype=torch.uint8), 'can_type': tensor([2])}
{'image': tensor([[[[255, 255, 255,  ..., 112, 130, 124],
          [255, 255, 255,  ..., 241, 237, 227],
          [255, 255, 255,  ..., 206, 201, 229],
          ...,
          [171, 148, 155,  ..., 196, 189, 217],
          [147, 156, 151,  ..., 237, 234, 170],
          [154, 155, 149,  ..., 186, 199, 189]]]], dtype=torch.uint8), 'can_type': tensor([1])}
{'image': tensor([[[[255, 255, 255,  ..., 121, 160, 127],
          [255, 255, 255,  ..., 114, 170, 185],
          [255, 255, 255,  ..., 123, 157, 226],
          ...,
          [154, 112, 125,  ..., 254, 254, 255],
          [137, 155,  97,  ..., 253, 254, 255],
          [16

In [64]:
for i in dataset:
    print(i)

{'image': tensor([[[[131, 142, 152,  ..., 247, 241, 255],
          [131, 125, 140,  ..., 248, 253, 249],
          [150, 117, 138,  ..., 254, 255, 255],
          ...,
          [255, 255, 255,  ..., 186, 196, 219],
          [255, 255, 255,  ..., 223, 195, 203],
          [253, 253, 253,  ..., 190, 202, 216]]]], dtype=torch.uint8), 'can_type': tensor([1])}
{'image': tensor([[[[100,  96,  89,  ..., 255, 255, 254],
          [ 86, 107, 150,  ..., 255, 255, 254],
          [197, 181, 132,  ..., 255, 255, 254],
          ...,
          [221, 238, 238,  ..., 196, 196, 200],
          [228, 245, 242,  ..., 201, 198, 195],
          [248, 229, 246,  ..., 198, 199, 201]]]], dtype=torch.uint8), 'can_type': tensor([1])}
{'image': tensor([[[[139, 137, 121,  ..., 255, 255, 254],
          [136, 145, 184,  ..., 255, 255, 254],
          [209, 238, 193,  ..., 255, 255, 254],
          ...,
          [181, 168, 198,  ..., 215, 201, 194],
          [189, 184, 220,  ..., 205, 198, 196],
          [20

KeyboardInterrupt: 

In [87]:
len(test_loader)

3

In [ ]:
d = []
for i in range(len(train_loader)):
    for j in range(len(train_loader[i][0]['image'])):
        d.append(train_loader[i][0]['image'][j].shape

In [26]:
for i in train_loader:
    print(i)

/home/iisers/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


{'image': tensor([[[199, 198, 197,  ..., 197, 205, 203],
         [198, 199, 199,  ..., 204, 208, 205],
         [195, 198, 200,  ..., 206, 209, 206],
         ...,
         [102, 124, 118,  ..., 135, 115, 131],
         [126, 119, 111,  ..., 137, 123, 150],
         [134, 121, 117,  ..., 131,  78, 135]],

        [[255, 255, 255,  ..., 255, 255, 255],
         [255, 255, 255,  ..., 255, 255, 255],
         [255, 255, 255,  ..., 255, 255, 255],
         ...,
         [206, 251, 124,  ..., 161, 130, 139],
         [216, 242, 220,  ..., 148, 194, 179],
         [146, 125, 181,  ..., 158, 142, 176]],

        [[214, 214, 213,  ..., 215, 213, 211],
         [214, 214, 214,  ..., 213, 212, 209],
         [214, 215, 215,  ..., 211, 211, 210],
         ...,
         [135, 121, 119,  ..., 183, 181,  93],
         [ 98, 100, 119,  ..., 157, 132, 139],
         [ 96, 105,  91,  ..., 163, 112, 168]],

        ...,

        [[ 89,  86, 128,  ..., 253, 253, 253],
         [102, 109, 122,  ..., 253,

In [33]:
for i in dataflow['test']:
    print(type(i))

<class 'dict'>
<class 'dict'>


In [103]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = HybridModel().to(device)
model_without_qf = HybridModel_without_qf().to(device)
n_epochs = 5
optimizer = optim.Adam(model.parameters(), lr=5e-3, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=n_epochs)
criterion = torch.nn.CrossEntropyLoss()

In [89]:
for batch_idx, (data, targets) in enumerate(train_loader):
    print(data.shape)

for batch_idx, (data, targets) in enumerate(train_loader):
    print(targets)
    


ValueError: not enough values to unpack (expected 2, got 1)

In [90]:
for epoch in range(n_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        data, targets = data.to(device), targets.to(device)
        
        #forward
        outputs = model(data)
        loss = criterion(outputs, targets)

        losses.append(loss.item())


        #backward
        optimizer.zero_grad()
        loss.backward()

        #gradient descent or adam step
        optimizer.step()
        
        
    print("Epoch {} Batch {} Loss {:.4f}".format(epoch, batch_idx, loss.item()))


def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

    model.train()

print("Checking accuracy on training set")
check_accuracy(train_loader, model)
#
# \print("Checking accuracy on validation set")
#check_accuracy(val_loader, model)
print("Checking accuracy on test set")
check_accuracy(test_loader, model)    


ValueError: not enough values to unpack (expected 2, got 1)

In [74]:
dataflow['train'][0]['image'].shape

torch.Size([1, 1, 256, 256])

In [60]:
for i in dataflow['train']:
    print(i)

{'image': tensor([[ 16,  35,  77,  ..., 110, 115, 109],
        [ 60,  28,  65,  ...,  92,  86,  87],
        [ 51,  56,  65,  ..., 113,  73,  73],
        ...,
        [185, 184, 184,  ..., 177, 178, 179],
        [185, 185, 184,  ..., 177, 178, 179],
        [186, 185, 184,  ..., 177, 178, 179]], dtype=torch.uint8), 'can_type': tensor([2])}
{'image': tensor([[145, 145, 145,  ..., 151, 156, 152],
        [145, 144, 144,  ..., 155, 152, 152],
        [146, 145, 144,  ..., 146, 146, 154],
        ...,
        [102, 102, 101,  ...,  37,  86,  93],
        [ 95, 102, 100,  ...,  90, 104,  99],
        [ 92, 101, 105,  ..., 102,  99,  98]], dtype=torch.uint8), 'can_type': tensor([2])}
{'image': tensor([[237, 238, 238,  ..., 218, 219, 219],
        [238, 238, 238,  ..., 219, 220, 220],
        [238, 238, 238,  ..., 219, 220, 221],
        ...,
        [153, 181, 208,  ..., 144, 137, 137],
        [150, 216, 193,  ..., 142, 118, 125],
        [148, 184, 169,  ..., 116,  99, 129]], dtype=torc

In [66]:
for feed_dict in dataflow['train']:
    print(feed_dict['image'].shape)

torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1, 1, 256, 256])
torch.Size([1,

In [70]:
inputs = feed_dict['image'].to(device)
inputs.shape

torch.Size([1, 1, 256, 256])

In [90]:
outputs = model(inputs)

IndexError: too many indices for tensor of dimension 2

In [95]:
inputs = feed_dict['image'].to(device)
inputs.shape

torch.Size([1, 1, 256, 256])

In [104]:
accu_list1 = []
loss_list1 = []
accu_list2 = []
loss_list2 = []

def train(dataflow, model, device, optimizer):
    for feed_dict in dataflow['train']:
        inputs = feed_dict['image'].to(device)
        targets = feed_dict['can_type'].to(device)

        outputs = model(inputs)
        loss = F.nll_loss(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f"loss: {loss.item()}", end='\r')


def valid_test(dataflow, split, model, device, qiskit=False):
    target_all = []
    output_all = []
    with torch.no_grad():
        for feed_dict in dataflow[split]:
            inputs = feed_dict['image'].to(device)
            targets = feed_dict['can_type'].to(device)

            outputs = model(inputs, use_qiskit=qiskit)

            target_all.append(targets)
            output_all.append(outputs)
        target_all = torch.cat(target_all, dim=0)
        output_all = torch.cat(output_all, dim=0)

    _, indices = output_all.topk(1, dim=1)
    masks = indices.eq(target_all.view(-1, 1).expand_as(indices))
    size = target_all.shape[0]
    corrects = masks.sum().item()
    accuracy = corrects / size
    loss = F.nll_loss(output_all, target_all).item()

    print(f"{split} set accuracy: {accuracy}")
    print(f"{split} set loss: {loss}")

    return accuracy, loss

for epoch in range(1, n_epochs + 1):
    # train
    print(f"Epoch {epoch}:")
    train(dataflow, model, device, optimizer)
    print(optimizer.param_groups[0]['lr'])

    # valid
    accu, loss = valid_test(dataflow, 'test', model, device, )
    accu_list1.append(accu)
    loss_list1.append(loss)
    scheduler.step()


Epoch 1:


IndexError: index 1 is out of bounds for dimension 1 with size 1

In [28]:
for i in dataset:
    print(i)

{'image': array([[131, 142, 152, ..., 247, 241, 255],
       [131, 125, 140, ..., 248, 253, 249],
       [150, 117, 138, ..., 254, 255, 255],
       ...,
       [255, 255, 255, ..., 186, 196, 219],
       [255, 255, 255, ..., 223, 195, 203],
       [253, 253, 253, ..., 190, 202, 216]], dtype=uint8), 'can_type': tensor([1])}
{'image': array([[100,  96,  89, ..., 255, 255, 254],
       [ 86, 107, 150, ..., 255, 255, 254],
       [197, 181, 132, ..., 255, 255, 254],
       ...,
       [221, 238, 238, ..., 196, 196, 200],
       [228, 245, 242, ..., 201, 198, 195],
       [248, 229, 246, ..., 198, 199, 201]], dtype=uint8), 'can_type': tensor([1])}
{'image': array([[139, 137, 121, ..., 255, 255, 254],
       [136, 145, 184, ..., 255, 255, 254],
       [209, 238, 193, ..., 255, 255, 254],
       ...,
       [181, 168, 198, ..., 215, 201, 194],
       [189, 184, 220, ..., 205, 198, 196],
       [207, 153, 166, ..., 195, 198, 196]], dtype=uint8), 'can_type': tensor([1])}
{'image': array([[255,

In [ ]:
dataflow

In [30]:
for feed_dict in dataflow['train']:
    print(feed_dict)

{'image': tensor([[[110, 215, 254,  ..., 250, 254, 255],
         [130, 231, 255,  ..., 136, 250, 253],
         [165, 248, 253,  ..., 171, 205, 248],
         ...,
         [137, 144, 136,  ...,  51, 109, 220],
         [167, 172, 114,  ..., 101,  97, 106],
         [248, 185, 204,  ...,  75, 109, 111]],

        [[193, 193, 192,  ...,  63,  87, 104],
         [193, 193, 192,  ...,  68,  65,  93],
         [193, 193, 192,  ...,  79,  60,  81],
         ...,
         [187, 187, 187,  ...,  53,  69,  85],
         [187, 187, 186,  ...,  76,  62,  67],
         [187, 186, 186,  ..., 102,  59,  60]],

        [[255, 255, 254,  ..., 244, 243, 241],
         [255, 255, 254,  ..., 244, 244, 243],
         [255, 255, 255,  ..., 244, 244, 243],
         ...,
         [141, 118,  89,  ..., 227, 227, 227],
         [156, 121, 122,  ..., 225, 225, 226],
         [189, 150, 107,  ..., 225, 225, 225]],

        ...,

        [[234, 234, 234,  ..., 218, 217, 216],
         [235, 234, 234,  ..., 218,